Requirement not upgraded as not directly required: ipykernel in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from jupyter_console) (4.8.2)
    100% |████████████████████████████████| 348kB 29.8MB/s ta 0:00:01
Requirement not upgraded as not directly required: ipython in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from jupyter_console) (6.4.0)
Requirement not upgraded as not directly required: jupyter-client in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from jupyter_console) (5.2.3)
Requirement not upgraded as not directly required: pygments in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from jupyter_console) (2.2.0)
Requirement not upgraded as not directly required: traitlets>=4.1.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from ipykernel->jupyter_console) (4.3.2)
Requirement not upgraded as not directly required: tornado>=4.0 in /home/ec2-user/anaconda3/envs/python3/

In [2]:
! pip install neo4j
from neo4j import GraphDatabase
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np


    100% |████████████████████████████████| 184kB 32.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for neo4j ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c0/dc/2e/6a26c3a5c63bb49a81092caa5ee48895971c0f229ee7900e7d
  Running setup.py bdist_wheel for neobolt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/54/00/270de525f8f966d864bd3a0224d38dca2573b53c1a54267720
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neo4j neobolt neotime
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

In [6]:
with driver.session() as session:
    result = session.run("MATCH (a:User) RETURN a.id LIMIT 10")
    

In [8]:
result.data()

[{'a.id': 'hG7b0MtEbXx5QzbzE6C_VA'},
 {'a.id': 'yXQM5uF2jS6es16SJzNHfg'},
 {'a.id': 'n6-Gk65cPZL6Uz8qRm3NYw'},
 {'a.id': 'dacAIZ6fTM6mqwW5uxkskg'},
 {'a.id': 'ssoyf2_x0EQMed6fgHeMyQ'},
 {'a.id': 'w31MKYsNFMrjhWxxAb5wIw'},
 {'a.id': 'jlu4CztcSxrKx56ba1a5AQ'},
 {'a.id': 'd6xvYpyzcfbF_AZ8vMB7QA'},
 {'a.id': 'sG_h0dIzTKWa3Q6fmb4u-g'},
 {'a.id': 'nMeCE5-xsdleyxYuNZ_7rA'}]

In [29]:
graph.run("CALL db.schema()").data()


[{'nodes': [(_-22:Category {constraints: [], indexes: ['name'], name: 'Category'}),
   (_-23:User {constraints: ['CONSTRAINT ON ( user:User ) ASSERT user.id IS UNIQUE'], indexes: [], name: 'User'}),
   (_-24:Review {constraints: ['CONSTRAINT ON ( review:Review ) ASSERT review.id IS UNIQUE'], indexes: [], name: 'Review'}),
   (_-25:Reputation {constraints: [], indexes: [], name: 'Reputation'}),
   (_-21:Business {constraints: ['CONSTRAINT ON ( business:Business ) ASSERT business.id IS UNIQUE'], indexes: [], name: 'Business'})],
  'relationships': [(Review)-[:REVIEWS {}]->(Business),
   (User)-[:WROTE {}]->(Review),
   (User)-[:FRIEND {}]->(User),
   (User)-[:COOL_LEVEL {}]->(Reputation),
   (User)-[:FUNNY_LEVEL {}]->(Reputation),
   (Business)-[:IN_CATEGORY {}]->(Category)]}]

now trying to automate atleast some of the process for the other attributes

| qunatile | ident | u.cool | u.funny | u.useful |
| --- | --- | --- | --- | --- |
| 0.20 | 1072521.40	| 0.0 | 0.0 | 0.0 |
| 0.35 | 1968091.95 | 0.0	 |0.0	|1.0 |
| 0.50 |	3063662.50	| 0.0	 | 0.0	| 3.0 | 
| 0.65 |	4389233.05	| 1.0	 | 1.0	| 7.0 |
| 0.80 |	5984803.60	| 5.0	 | 4.0	| 19.0 |
| 0.95 |	7830374.15	| 36.0	| 33.0	| 110.0 |


In [24]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a funny value.
# graph.run("""
# MATCH (n)
# WHERE ID(n)= 6543
# MATCH (u:User) WHERE NOT exists(u.funny)
# CREATE (u)-[:FUNNY_LEVEL]->(n)
# """)

In [ ]:
#this is for testing the funny edge adding loop
id_list=[(6543,0,1),(6668,1,2),(6552,2,33),(6561,33,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}=<u.funny<{index[2]}\
    RETURN n.funny, u.name, u.funny LIMIT 6").to_data_frame()
    dflist.append(tempdf)

In [25]:
#this is for adding the funny edges
# id_list=[(6668,1,2),(6552,2,33),(6561,33,999999)]
# dflist=[]



# for index in id_list:
#     tempdf=graph.run(f"\
#     MATCH (n)\
#     WHERE ID(n)={index[0]}\
#     MATCH (u:User) WHERE {index[1]}<=u.funny<{index[2]}\
#     CREATE (u)-[:FUNNY_LEVEL]->(n)").to_data_frame()
#     dflist.append(tempdf)


In [34]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a useful value.
# graph.run("""
# MATCH (n)
# WHERE ID(n)= 6670
# MATCH (u:User) WHERE NOT exists(u.useful)
# CREATE (u)-[:USEFUL_LEVEL]->(n)
# """)

In [32]:
#this is for testing the funny edge adding loop
id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(6683,110,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
    RETURN n.useful, u.name, u.useful LIMIT 6").to_data_frame()
    dflist.append(tempdf)

In [37]:
dflist;

In [38]:
#this is for adding the useful edges
# id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(6683,110,999999)]
# dflist=[]



# for index in id_list:
#     tempdf=graph.run(f"\
#     MATCH (n)\
#     WHERE ID(n)={index[0]}\
#     MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
#     CREATE (u)-[:USEFUL_LEVEL]->(n)").to_data_frame()
#     dflist.append(tempdf)


maybe put ghost users back in graph?

In [ ]:
UNWIND split(friends,',') as friend
MERGE (u1:User{id:friend})
MERGE (u)-[:FRIEND]-(u1)